In [ ]:
import polars as pl
from Levenshtein import ratio

Constants

In [ ]:
RATIO = .7 # similarity between T&Cs and gateway list
PARTIAL_RATIO = .5

In [ ]:
terms = (
    pl.scan_csv("data/terms.csv", infer_schema_length=10000) 
)

In [ ]:
gateway = (
    pl.scan_csv('data/gateway_connections.csv', infer_schema_length=10000)
    .with_columns(
        pl.col('Licensee').str.replace_all(r'\([^)]*\)', '').str.strip_chars(' )').str.to_uppercase().alias('licensee_split'),
    )
)


In [ ]:
combo = (
    terms
    .join(
        gateway, left_on='org_name', right_on='Licensee', how='outer'
    )
    .drop(
        'Address', 'State(s) Located In' , 'Business Type', 'Website', 'file',
        'Date Requested',' Last Updated', '# of Prescribers', '# of Pharmacists', 'Last Updated'
    )
    .drop_nulls()
)
combo.collect()

In [ ]:
combo.collect().write_excel(
    'data/test.xlsx',
    worksheet='match',
    autofit=True,
    freeze_panes=((1,0,0,0))
)